In [ ]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI 
from pathlib import Path
from pydantic import BaseModel, Field
from agents import Agent, Runner,  AgentOutputSchema
from agents import Agent, Runner,  AgentOutputSchema
from src.tools.schema import ExperimentSpec
import json 
from agents.tool import WebSearchTool, FileSearchTool
from tools.fileWriter import write_file_tool
from tools.executeCommand import execute_command_tool
from tools.fileReader import read_file_tool
import subprocess
load_dotenv()
llm = ChatOpenAI(model="gpt-5.2", temperature=0)

In [ ]:
INSTRUCTIONS = Path('AgentPrompts/parser.txt').read_text(encoding="utf-8")
parser = Agent(
    name = 'Experiment Specification Parser', 
    instructions=INSTRUCTIONS,
    output_type= AgentOutputSchema(ExperimentSpec, strict_json_schema=False),
    
    model = 'gpt-4o'
)
user_text = input('''
                  
Describe the experiment you want to run.
                  
Please explain, in plain English:
    - What the experiment is doing conceptually (what is being simulated, evaluated, or tested)
    - What inputs can vary (parameters you want to sweep or optimize, with ranges or options)
    - What inputs stay fixed (constants or conditions held the same across runs)
    - What your goal is, if any (explore behavior, minimize/maximize something, hit a target)
    - What you want to measure (outputs or metrics you care about)
    - How you want the system to search (exhaustive grid, random sampling, or adaptive optimization)
    - Where it should run (local machine, cluster, container)
    - Any important setup details, assumptions, or context?
                  
''')
parser_result = await Runner.run(parser, user_text)
spec = parser_result.final_output 
data = spec.model_dump_json(indent=2)

with open("experiment.json", "w") as f:
    f.write(data)

In [ ]:
INSTRUCTIONS = Path('AgentPrompts/critic.txt').read_text(encoding="utf-8")
spec = ExperimentSpec.model_validate(
    json.loads(Path("experiment.json").read_text())
)
critic = Agent(
    name = 'Spec Critic',
    instructions = INSTRUCTIONS,
    model = 'gpt-4o',
)
critic_result = await Runner.run(critic, spec.model_dump_json(indent=2))
user_clarifying_response = input(critic_result.final_output)

In [ ]:
editor_partial_instructions = Path("AgentPrompts/editor.txt").read_text(encoding="utf-8")
INSTRUCTIONS = '''
    You are an experiment specification editor.

    Task:
    Given:
        (1) an existing experiment specification JSON:''' + spec.model_dump_json(indent=2) + ''''
        (2) clarification questions that were asked to the user: ''' + critic_result.final_output +  '''
        (3) the user’s answers to those questions, which will be provided to you as input
    produce an UPDATED experiment specification JSON.
''' + editor_partial_instructions

editor = Agent(
    name = "Experiment Spec Editor",
    instructions=INSTRUCTIONS, 
    model = 'gpt-4o',
    output_type= AgentOutputSchema(ExperimentSpec, strict_json_schema=False),
)
editor_result = await Runner.run(editor, user_clarifying_response)
spec = editor_result.final_output 
data = spec.model_dump_json(indent=2)

with open("experiment.json", "w") as f:
    f.write(data)

In [ ]:
INSTRUCTIONS = Path("AgentPrompts/summarizer.txt").read_text(encoding="utf-8")
summarizer = Agent(
    name ='Experiment Summerizer',
    instructions = INSTRUCTIONS, 
    model = 'gpt-4o'
)
spec = ExperimentSpec.model_validate(
    json.loads(Path("experiment.json").read_text())
)
summarizer_result = await Runner.run(summarizer, spec.model_dump_json(indent=2))
print("Here is your experiment as I understand it:")
print(summarizer_result.final_output)
print("Now, I will conduct the appropriate research needed to conduct your experiment.")

In [8]:
INSTRUCTIONS = Path("AgentPrompts/researcher.txt").read_text(encoding="utf-8")
spec = ExperimentSpec.model_validate(
    json.loads(Path("experiment.json").read_text())
)
researcher = Agent(
    name ='Experiment Researcher',
    instructions = INSTRUCTIONS, 
    model = 'gpt-4o',
    tools= [WebSearchTool()]
)
researcher_result = await Runner.run(researcher, spec.model_dump_json(indent=2))

with open("research.txt", "w") as f:
    f.write(researcher_result.final_output)

In [ ]:
INSTRUCTIONS = Path("AgentPrompts/executor.txt").read_text(encoding="utf-8")
executer = Agent(
    name = 'Experiment Executor',
    instructions = INSTRUCTIONS,
    model = 'gpt-4o',
    tools = [
        write_file_tool, 
        execute_command_tool
    ]
)
executor_result = await Runner.run(executer, researcher_result.final_output)
print(executor_result.final_output)


In [ ]:
INSTRUCTIONS = Path("AgentPrompts/plotter.txt").read_text(encoding="utf-8")
plotter = Agent(
    name = 'Results Plotter',
    instructions = INSTRUCTIONS,
    model = 'gpt-4o',
    tools = [
        read_file_tool, 
        write_file_tool,
        execute_command_tool]
)
plot = await Runner.run(plotter, "results.log")
print(plot.final_output)



In [11]:
INSTRUCTIONS = Path("AgentPrompts/writer.txt").read_text(encoding="utf-8")
writer = Agent(
    name = 'Report Writer', 
    instructions = INSTRUCTIONS,
    tools = [
        write_file_tool,
        read_file_tool, 
    ]
)
files = "results.log, experiment.json, research.txt, plot.png, " + executor_result.final_output
report = await Runner.run(writer, files)